# Parser 9.30 - Increment and Decrement

Here we intend to implement [C-style](https://en.wikipedia.org/wiki/C_(programming_language)) **'++'** increment and **'--'** decrement operators. When used, they increase or decrease the value of a variable by one. It is an error to try to apply them to numbers.

These unary operators come in two forms, prefix and postfix, depending on whether they appear before or after the variable they apply to. Both forms have the same effect on the variable they are applied to, but the value of the expression differs slightly. The result of the prefix form is the changed value of the variable *after* increment or decrement. The result of the postfix form is the original value of the variable *before* increment or decrement.

In many practical cases the distinction means little. Arguably the majority of expressions involving these operators consist solely of a variable and one of these operators. The real goal is merely to change the value of that variable; the expression result is simply ignored.

However in larger expressions, as when perhaps assigning the result to another variable, the difference can very well matter and should be taken into account.

## Libraries

In [28]:
import glob       # for searching directories
import math       # for 'log--()' functions

import re         # for regular expressions

## User output

In [29]:
visSep = '-------------'             # visual separator

def UIwriteln(this):
    '''write a single line to output'''
    print( f'{this}\n' )
    
def UIwriteSep():
    '''write a visual separator'''
    UIwriteln( visSep )

def UIshow(tag, value):
    '''write a tagged value to output'''
    UIwriteln( f'{tag}: {value}' )

def UIerror(this):
    '''write an error message to output'''
    UIshow( 'Error', this )

# Tracing

In [30]:
# flags: show trace of processing

showInteract = True          # default for interactive use
showBatch = False            # default for batch use

showTrace = None             # control flag

# Trace Output

def TOshow(mesg, text):
    '''write trace message to output if enabled'''
    if showTrace:
        UIshow( f'{mesg:15s}', text )
        
def TOstring(tag, this):
    
    if showTrace:
        TOshow( tag, ' '.join([str(e) for e in this]) )

# -----------------------
# Parse Tracing
# -----------------------

def PTshowexpr(this):

    TOshow( 'Parse', visSep )
    TOshow( 'Current Expr', this )

def PTshowparse(ok, res, opStk, typStk):

    if ok:
        TOstring( 'Current RPN', res )
        TOstring( 'Operator Stack', opStk )
        TOstring( 'Type Stack', typStk )

def PTshowtoken(this):

    if not this[0] == ' ':
        TOshow( "Found Token", this )

# -----------------------
# Evaluation Tracing
# -----------------------

def ETshowtoken(this):
    
    TOshow( 'Eval', visSep )
    TOshow( 'Current token', this )

def ETshoweval(stk):
    
    TOstring( 'Operand Stack', stk )


# Parser

In [31]:
# operands accepted:
# - decimal and hexadecimal floating point literals
# - scalar numeric variables

# operators accepted:
# - unary negation, plus
# - binary addition, subtraction, multiplication, division
# - grouping parentheses
# - logical not, equality, inequality
# - assignment and shortcut assignment
# - prefix and postfix increment and decrement

# errors detected:
# - unrecognized input
# - out of range numeric input
# - malformed expression

# result tuple:
# - (True, [parse])
# - (False, None)

class Parser(object):
    
    VERSIONNUMBER = '9.30'
    
    _FLTMAX =  4294967295                                  # 2**32-1
    _FLTMIN = -4294967296                                  # -(2**32)

    _expMax = {
        'P' : math.log2(_FLTMAX),                          # max base 2 exponent
        'E' : math.log10(_FLTMAX)                          # max base 10 exponent
    }

    _typeChkLst = {                                         # type check control lists
         'n2n': ['number', 'number'],                       # - index is left-to-right
         'v2n': ['number', 'numsym'],
        'nn2n': ['number', 'number', 'number'],             # check is right-to-left
        'vn2n': ['number', 'numsym', 'number']
    }
    
    def __init__(self):
        pass
        
    def doparse(self, this):

        def parseErr(mesg, pos):
            '''report parse error'''
            UIerror(mesg)
            UIwriteln(f'>>> {this}')
            if pos > 0:
                UIwriteln(f'{"^^near here".rjust(pos)}')
            return False

        # initialize

        expr = this                # save to new variable but retain original for error reports
        start = 15                 # tracked so we can report where in an expression an error occurred
        token = None               # anything successfully matched
        ok = wantoperand = True    # flags
        result = []                # rpn expression
        opStk = [ ('EOE', 1) ]     # operator stack
        typStk = []                # type stack
        

        def typeCheck(op, chk):
            '''type check operands'''            
            check = list(self._typeChkLst[chk])       # list() to avoid aliasing
            while len(check) > 1:
                want = check.pop()
                have, rpnPos, errPos = typStk.pop()
                              
                # do we want a number ?
                
                if want == 'number':
                    
                    # convert symbols to numbers
                    
                    if have == 'numsym':
                        result.insert( rpnPos, 'U*' )
                        
                # do we want a variable ?
                
                elif want == 'numsym':
                    
                    if have != 'numsym':
                        return parseErr( 'Numeric variable required', errPos )
                        
            # push intermediate result type (no position)
                    
            typStk.append( (check[0], -1, errPos) )
            return True
 
        def addOperand(op, typ):
            '''add operand to RPN'''
            result.append( op )
            typStk.append( (typ, len(result), start) )
            
        def addOperator():
            '''add operator to RPN'''            
            op, _, chk = opStk.pop()
            result.append( op )
            return typeCheck( op, chk )
            
        def popGEop(prec):
            '''pop operators of equal or greater precedence'''
            ok = True
            while ok and prec <= opStk[-1][1]:
                ok = addOperator()
            return ok
 
        def pushLeft(op, prec, chk):
            '''push left associative operator on stack'''
            if not popGEop(prec):
                return False
            opStk.append( (op, prec, chk) )
            return True

        def popGop(prec):
            '''pop operators of greater precedence'''
            ok = True
            while ok and prec < opStk[-1][1]:
                ok = addOperator()
            return ok
 
        def pushRight(op, prec, chk):
            '''push right associative operator on stack'''
            if not popGop(prec):
                return False
            opStk.append( (op, prec, chk) )
            return True

        def popUntil(op, prec):
            '''clear and check operator stack'''
            if not popGEop(prec):
                return False
            elif op == opStk.pop()[0]:
                return True
            elif op == '(':
                return parseErr('Unmatched right parenthesis', start)
            elif op == 'EOE':
                return parseErr('Unmatched left parenthesis', 0)      
    
        # operator dictionaries initialization
        
        _postOps = '[+]{2}|[-]{2}'
        
        _postOp = {
            '++': (90, pushLeft, 'v2n'),
            '--': (90, pushLeft, 'v2n')
        }

        _unOps = '!|[+]{1,2}|[-]{1,2}'

        _unOp =  {
            '-':  (80, pushRight, 'n2n'),
            '+':  (80, pushRight, 'n2n'),
            '!':  (80, pushRight, 'n2n'),
            '++': (80, pushRight, 'v2n'),
            '--': (80, pushRight, 'v2n')
        }

        _binOps = '[-+*/]=?|[=!]?='

        _binOp = {
            '*':  (70, pushLeft,  'nn2n'),
            '/':  (70, pushLeft,  'nn2n'),
            '+':  (60, pushLeft,  'nn2n'),
            '-':  (60, pushLeft,  'nn2n'),
            '==': (50, pushLeft,  'nn2n'),
            '!=': (50, pushLeft,  'nn2n'),
            '=':  (10, pushRight, 'vn2n'),
            '*=': (10, pushRight, 'vn2n'),
            '/=': (10, pushRight, 'vn2n'),
            '+=': (10, pushRight, 'vn2n'),
            '-=': (10, pushRight, 'vn2n')
        }
      
        def convertFloat(fplit, base, capgrp):
            '''convert floating point literal to internal form'''
            
            def rangeErr():
                return parseErr(f'\'{fplit}\' is out of range', start)
            
            # collect the features of interest
                   
            p = re.search(capgrp, fplit.upper())
                            
            lint, lfrc, expbas, expsgn, lexp = p.group(1,2,4,5,6)
            
            # convert integer portion (if any)
 
            uint = 0
            if lint:
                p = re.search('[1-9A-F][0-9A-F]*', lint )
                if p != None:
                    for ch in p.group():
                        digval = '0123456789ABCDEF'.find(ch)
                        if uint <= (self._FLTMAX - digval)/base:
                            uint =  uint * base + digval
                        else:
                            return rangeErr()
                    
            # convert fractional portion (if any)
                    
            ufrc = 0
            if lfrc:
                fbase = 1
                for ch in lfrc:
                    digval = '0123456789ABCDEF'.find(ch)
                    fbase *= base
                    ufrc += digval/fbase
        
            if uint == self._FLTMAX and ufrc != 0:
                return rangeErr()
            
            # value so far
            
            uflt = uint + ufrc
 
            # convert exponent portion (if any)
            
            uexp = 0
            if lexp:
                for ch in lexp:
                    digval = '0123456789'.find(ch)
                    if uexp <= (self._expMax[expbas] - digval)/10:
                        uexp =  uexp * 10 + digval
                    else:
                        return rangeErr()
                    
            # adjust value by exponent (if any)
             
            if uexp:
                power = (2 if expbas == 'P' else 10) ** uexp
                if expsgn == '-':
                    uflt /= power
                elif uflt <= self._FLTMAX/power:
                    uflt *= power
                else:
                    return rangeErr()
                    
            addOperand( uflt, 'numlit' )
            return True

        def startsWith(regex):
            '''test if expression starts with given regular expression'''
            nonlocal expr, start, token

            p = re.match(regex, expr)
            if p == None:
                return False
            else:
                token = p.group()              # what we matched
                start += len(token)            # update to next match position in original string
                expr = expr[len(token):]       # "chop off" what we matched
                PTshowtoken(token)             # trace
                return True

        # top level main loop
              
        while ok and len(expr):

            _ = startsWith('[ ]+')                             # skip leading whitespace

            PTshowexpr(expr)                                   # trace

            # look for operand

            if wantoperand:

                if startsWith('[(]'):
                    '''left parenthesis ?'''
                    opStk.append( ('(', 2) )

                elif startsWith(_unOps):
                    '''right unary?'''
                    prec, assoc, check = _unOp[token]
                    assoc( 'U' + token, prec, check )

                else:

                    wantoperand = False                         # flip
                    
                    if startsWith('0[xX]([0-9a-fA-F]+([.][0-9a-fA-F]*)?|[.][0-9a-fA-F]+)([pP][-+]?[0-9]+)?'):
                        '''unsigned hexadecimal literal ?'''
                        ok = convertFloat(token, 16, '0X([0-9A-F]*)[.]?([0-9A-F]*)(([P])([-+])?([0-9]+))?')

                    elif startsWith('([0-9]+([.][0-9]*)?|[.][0-9]+)([eE][-+]?[0-9]+)?'):
                        '''unsigned decimal literal?'''
                        ok = convertFloat(token, 10, '([0-9]*)[.]?([0-9]*)(([E])([-+])?([0-9]+))?')
                        
                    elif startsWith('[a-zA-Z][_a-zA-Z0-9]*'):
                        '''numeric scalar variable?'''
                        addOperand( token.upper(), 'numsym')
                        
                    else:
                        '''malformed'''
                        ok = parseErr('Expecting operand', start)

            # look for operator

            else:

                if startsWith('[)]'):
                    ok = popUntil( '(', 4 )
                    
                elif startsWith(_postOps):
                    '''postfix inc/dec?'''
                    prec, assoc, check = _postOp[token]
                    ok = assoc( 'P' + token, prec, check )

                else:

                    wantoperand = True                                # flip
                    
                    if startsWith(_binOps):
                        '''binary operator?'''
                        prec, assoc, check = _binOp[token]
                        ok = assoc( 'B' + token, prec, check )

                    else:
                        '''malformed'''
                        ok = parseErr('Expecting operator', start)

            PTshowparse(ok, result, opStk, typStk )               # trace
 
        if ok and wantoperand:
            ok = parseErr('Unexpected end of expression', start ) # must be in 'wantoperator' state   

        if ok:
            ok = popUntil( 'EOE', 3 )                             # clear operator stack
            
        if ok:
            ok = typeCheck('LoneSymbol', 'n2n')                   # make sure final result is a number

        return (ok, result if ok else None)                       # done
       

### How it works

Recognizing the prefix form really is about as simple as we could hope for. They are right-associative unary operators with the same precedence as our existing unary operators. The only difference is in the type check. All we need do is add all this to the unary operator dictionary *_unOp{}* and modify the regex *_unOps* to match them:

```Python
'!|[+]{1,2}|[-]{1,2}'
```

The [quantifier](https://www.geeksforgeeks.org/perl-quantifiers-in-regular-expression/) *{1,2}* is applied to both **'+'** and **'-'**, which allows them to match any of:

```Python
+ ++ - --
```

So the whole regex matches a **'!'** character *or* one or two **'+'** characters *or* one or two **'-'** characters.

>We placed **'+'** in a character class to "turn off" its [regex metachar](https://docs.python.org/3/howto/regex.html#matching-characters) meaning (placing **'-'** in a class is just for symmetry). Note the metachar meaning of **'+'** can be considered a shorthand for the quantifier *{1,}*.

We don't need to modify the *typeCheck()* function itself. We do add another entry, *'v2n'*, to *_typeChkLst{}* to tell it what we want it to do: verify our new operators are being applied to a variable.

And that's its. The parser can now handle prefix increment and decrement operators appropriately.

Things are not quite so simple for postfix increment and decrement. To start with, they are left-associative and have a higher precedence than the prefix forms. The unary operator dictionary wouldn't have any problem accomodating that, however. What we really need to deal with is that the postfix forms must be recognized in the *wantoperator* state (which means adding code) and that they are identical to the prefix forms (**'++'** and **'--'** are *polymorphic* operators, our first example since **'-'**).

For the postfix forms we create a new dictionary, *_postOp{}*, and another regex to recognize them, *_postOps*. This might be considered overkill. For now we could have written the necessary check simply as:

```Python
elif startsWith('[+]{2}|[-]{2}'):
    pushLeft( 'P' + token, 90, 'v2n' )
```

and been just fine. Instead we actually wrote:

```Python
elif startsWith(_postOps):
    '''postfix inc/dec?'''
    prec, assoc, check = _postOp[token]
    ok = assoc( 'P' + token, prec, check )
```

Why be more verbose? Mainly because reating all operators in basically the same way means not ever having to remember that some are handled differently than others. Which is especially helpful when the need for modifications becomes apparent.

>Also serendipity. Trying to treat all operators the same way sometimes leads to noticing that it's not only possible to do so, but overall more esthetically pleasing to.

>Why *P* as a prefix for postfix operators? Simply because we needed some way to distinguish between postfix and prefix forms in the Reverse Polish and *P* is just what happened to come to mind first.

# Evaluator

In [32]:
# operators handled:
# - unary negation, plus
# - binary addition, subtraction, multiplication, division
# - logical negation, equality and inequality
# - variable name de-reference
# - variable assignment and shortcut assignment
# - prefix and postfix increment and decrement

# errors detected:
# - out of range
# - division by zero

# return tuple:
# - (True, result)
# - (False, None)

class Evaluator(Parser):
    
    def __init__(self):
        self._symTable = dict()
 
    def doeval(self, rpn):
 
        def inRange(ok, val):
            '''range check test result'''
            if ok:
                stk.append( val )
            else:
                UIerror( 'Evaluation result out of range' )
            return ok
        
        def pushOperand(val):
            '''push operand on stack'''
            stk.append( val )
            return True
                     
        def binVal(val, var):
            '''assign value to variable'''
            self._symTable[var] = val
            return pushOperand(val)
        
        def unVal(var):
            '''variable name value'''
            return pushOperand(self._symTable[var] if var in self._symTable else 0)

#        def unPlu(arg):
#            '''unary plus'''
#            return pushOperand( arg )

        def unNeg(arg):
            '''unary negation'''
            return inRange( arg != self._FLTMIN, -arg )
        
        def unNot(arg):
            '''logical not'''
            return pushOperand( not arg )

        def binAdd(rgt, lft):
            '''binary addition'''
            if lft >= 0:
                return inRange( rgt <= self._FLTMAX - lft, lft+rgt )       
            else:
                return inRange( rgt >= self._FLTMIN - lft, lft+rgt )
 
        def binSub(rgt, lft):
            '''binary subtraction'''
            if lft >= 0:
                return inRange( lft - self._FLTMAX <= rgt, lft-rgt )
            elif rgt >= 0:
                return inRange( lft - self._FLTMIN >= rgt, lft-rgt )
            else:
                return inRange( lft <= self._FLTMAX + rgt, lft-rgt )
 
        def binMul(rgt, lft):
            '''binary multiplication'''
            if abs(lft) <= 1 or abs(rgt) <= 1:
                return pushOperand( lft * rgt )

            elif lft > 0:
                if rgt > 0:
                    return inRange( rgt <= self._FLTMAX / lft, lft * rgt )
                else:
                    return inRange( rgt >= self._FLTMIN / lft, lft * rgt )

            elif rgt > 0:
                return inRange( rgt <= self._FLTMIN / lft, lft * rgt )
  
            else:
                return inRange( rgt >= self._FLTMAX / lft, lft * rgt )
 
        def binDiv(rgt, lft):
            '''binary division'''
            if abs(rgt) >= 1:
                return pushOperand( lft/rgt )
            
            elif rgt > 0:
                if lft > 0 :
                    return inRange( lft <= self._FLTMAX * rgt, lft / rgt )
                else:
                    return inRange( lft >= self._FLTMIN * rgt, lft / rgt )
            
            elif rgt < 0:
                if lft > 0:
                    return inRange( lft <= self._FLTMIN * rgt, lft / rgt )
                else:
                    return inRange( lft >= self._FLTMAX * rgt, lft / rgt )

            else:
                UIerror( 'Division by zero' )
                return False
            
        def binEqu(rgt, lft):
            '''logical equality'''
            return pushOperand( lft == rgt )
        
        def binNeq(rgt, lft):
            '''logical inequality'''
            return pushOperand( lft != rgt )
        
        def binShortVal(val, var, op):
            '''shortcut assignment'''
            _ = unVal(var)                           # put the value of the variable on the stack
            if op(val, stk.pop()):                   # perform the arithmetic
                return binVal(stk.pop(), var)        # if successful,  also assign result to variable
            return False
        
        def binAddVal(val, var):
            return binShortVal(val, var, binAdd)
        
        def binSubVal(val, var):
            return binShortVal(val, var, binSub)
        
        def binMulVal(val, var):
            return binShortVal(val, var, binMul)
        
        def binDivVal(val, var):
            return binShortVal(val, var, binDiv)
        
        def unPfxInc(var):
            return binShortVal(1, var, binAdd)
        
        def unPfxDec(var):
            return binShortVal(1, var, binSub)
        
        def unPostFix(val, var):
            '''postfix inc/dec'''
            _ = unVal(var)                             # push current value of variable on stack
            ok = binShortVal(val, var, binAdd)         # update value of variable
            stk.pop()                                  # remove updated value from stack (if error, removes first push)
            return ok
                
        def unPstInc(var):
            return unPostFix(1, var)
            
        def unPstDec(var):
            return unPostFix(-1, var)
        
        # initialize
                                
        unDispatch = {
            'U*': unVal,
            'U-': unNeg,
            'U+': pushOperand,
            'U!': unNot,
            'U++': unPfxInc,
            'U--': unPfxDec,
            'P++': unPstInc,
            'P--': unPstDec
        }
        
        binDispatch = {
            'B+': binAdd, 'B-': binSub,
            'B+=': binAddVal, 'B-=': binSubVal,
            'B*': binMul, 'B/': binDiv,
            'B*=': binMulVal, 'B/=': binDivVal,
            'B==': binEqu, 'B!=': binNeq,
            'B=': binVal
        }
  
        stk = []
        ok = True

        # main loop
        
        TOstring('Input', rpn)
        
        for v in rpn:

            ETshowtoken(v)
            
            if v in binDispatch:
                ok = binDispatch[v](stk.pop(), stk.pop())
                
            elif v in unDispatch:
                ok = unDispatch[v](stk.pop())
                
            else:
                stk.append( v )

            if not ok:
                return ( False, None )

            ETshoweval( stk )

        return ( True, stk.pop() )


### How it works

Because all our new operators are unary and the prefix and postfix operator forms are different, we can add them all to *unDispatch{}*.

The functions which implement the prefix forms are quite simple. Consider that these expressions are essentially equivalent:

```Python
a += 1 <=> ++a
z -= 1 <=> --z
```

Both prefix operators change the value of the variable by one and leave that value on the operand stack as their result. Since *binShortVal()* can do exactly that, we can use it again to implement *U++* and *U--*.

The postfix forms are not quite so easy. Somehow we have to leave the original value of the variable affected, not its changed one, on the operand stack. Perhaps the most efficient way to do this in terms of speed would avoid leveraging our existing functions because they'll require redundant stack manipulations.

So be it. We want the error checks. We save the original variable value on the operand stack, then again use *binShortVal()* to update it. If this succeeds, the new variable value is at the top of the stack. We discard it, exposing the original value. If the update did not succeed, then the original value gets discarded instead. We don't care because we're about to stop evaluation anyway.


## Running the parser

In [33]:
passCnt = failCnt = 0                       # most useful for test input files, but never any harm

myParser = myEvaluator = None               # where we keep instances of our classes

def startUp(flag):
    '''begin execution'''
    global passCnt, failCnt, showTrace
    global myParser, myEvaluator
    if not myParser:
        myParser = Parser()
    if not myEvaluator:
        myEvaluator = Evaluator()
    UIshow( 'Parser', myParser.VERSIONNUMBER )
    passCnt = failCnt = 0
    showTrace = flag
    
def shutDown():
    '''terminate execution'''
    UIwriteSep()
    UIshow( 'Pass', passCnt )
    UIshow( 'Fail', failCnt )
    
# run parser
        
def parseOne(this):
    '''parse/evaluate one expression'''
    global passCnt, failCnt
    UIwriteSep()
    neg = this[0] == '@'
    if neg:
        this = this[1:]
    UIshow( 'Input', this )
    ok, res = myParser.doparse( this )
    if ok:
        UIshow( 'Final Parse', res )
        ok, res = myEvaluator.doeval( res )
        if ok:
            UIshow( 'Final Eval', res )
        if neg:
            ok = not ok
    if ok:
        passCnt += 1
    else:
        failCnt += 1

## Interactive use

In [34]:
def parse():
    
    startUp(showInteract)
    while True:
        inp = input( 'Expression: ' )
        UIwriteln( '' )                      # looks better with a blank line here
        if inp.lower() == 'quit':
            break
        elif inp.strip():
            parseOne( inp )
    shutDown()

## Batch processing

In [35]:
testDir = '..\\ParserTest\\'            # directory holding test input files (empty string if same as notebook directory)

# convert current version number to match test file numbers
# - done this way so we can update only the version number and everything still works

def currNum():
    
    head = myParser.VERSIONNUMBER[:len(myParser.VERSIONNUMBER)-3]
    tail = myParser.VERSIONNUMBER[-2:]
    return f'{head:0>2}{tail}'

# make full path name to test file

def makePath(typ, num):
    return f'{testDir}{typ}{num}.txt'

# run one test

def runTest(this):
    
    UIwriteln(f'Parser {myParser.VERSIONNUMBER} vs {this[-12:-4]}')
    
    myEvaluator._symTable.clear()
    
    with open(this) as f:
        data = f.readlines()
    for line in data:
        test = line.strip()
        if test and test[0] != '#':         # skip blank and comment lines
            parseOne(test)
    
# run a test of current or specified version which should succeed
    
def good(num='curr'):
  
    startUp(showBatch)
    runTest(makePath('pass', currNum() if num == 'curr' else num))
    shutDown()
    
# run a test of current or specified version which should fail

def bad(num='curr'):
    
    startUp(showBatch)
    runTest(makePath('fail', currNum() if num == 'curr' else num))
    shutDown()
    
# run regression test against current and all previous test files

def regress():
            
    UIwriteln('PASS tests')
    
    startUp(showBatch)                       # must create objects before we can access variables inside them 
    currFn = makePath('pass', currNum())
    failed = []
    fnlist = glob.glob(f'{testDir}pass????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = failCnt
            runTest(fn)
            if atstart < failCnt:
                failed.append(fn)               
    shutDown()
    
    UIwriteln('FAIL tests')
    
    startUp(showBatch)
    currFn = makePath('fail',currNum())
    passed = []
    fnlist = glob.glob(f'{testDir}fail????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = passCnt
            runTest(fn)
            if atstart < passCnt:
                passed.append(fn)                
    shutDown()
    
    if not len(failed):
        UIwriteln('All pass tests succeded')
    else:
        UIwriteln('Pass tests which failed')
        for fn in failed:
            UIwriteln(f'  {fn}')
            
    if not len(passed):
        UIwriteln('All fail tests succeded')
    else:
        UIwriteln('Fail tests which passed')
        for fn in passed:
            UIwriteln(f'   {fn}')
              

# Testing the parser

In [ ]:
parse()       # interactive, one expression at a time

In [ ]:
good()        # run current parser against its own pass test. Use good('1234') to run against specific pass test.

In [ ]:
bad()         # run current parser against its own fail test. Use bad('5678') to run against specific fail test.

In [ ]:
regress()     # run parser against all previous tests